In [3]:
## Plasma Hamiltonian Simulation Resource Estimates

In [1]:
using Pkg
Pkg.activate("/home/gribeill/GitHub/HiQuER/")
using HiQuER

using Revise
using ProgressBars

macro bash_str(s) open(`bash`,"w",stdout) do io; print(io, s); end; end

  Activating project at `~/GitHub/HiQuER`


@bash_str (macro with 1 method)

In [45]:
ϵ = 1e-3

function analytic_bound(r::Int, k::Int, L::Int, Λ::Float64, t::Float64)
    
    A = BigFloat(2*L*5^(k-1)*Λ*abs(t))^(2k+1)/(3*BigInt(r)^BigInt(2k))
    return A * exp(BigFloat(2*L*5^(k-1)*Λ*abs(t))/BigFloat(r))
end

function minimize_bound(ϵ::Float64, k::Int, L::Int, Λ::Float64=1.0, t::Float64=1.0)
    r = 8
    ϵ_b = analytic_bound(r, k, L, Λ, t)
    iter = 1
    while ϵ_b > ϵ && iter < 128
        r = r*2
        ϵ_b = analytic_bound(r, k, L, Λ, t)
        iter +=1
    end
    
    r_left = Int(r//2)
    r_right = r
    
    ϵ_L = analytic_bound(r_left, k, L, Λ, t)
    ϵ_R = analytic_bound(r_right, k, L, Λ, t)
    r_M = 0
    while (r_right - r_left) > 2
        r_M = Int(r_left + (r_right - r_left)//2)
        ϵ_M = analytic_bound(r_M, k, L, Λ, t)
        
        if ϵ_M < ϵ
            r_right = r_M
        else
            r_left = r_M
        end
        
        #println("$r_left, $r_M, $r_right")
    end
        
    return r_M
end

minimize_bound (generic function with 3 methods)

In [48]:
empirical_bound(N) = 4.035* N ^ 1.555

empirical_bound (generic function with 1 method)

In [2]:
#Parameters

N_qubit = [2^n for n=3:10];
k = 2
α = 0.6
Tmax = 1.0; #maximum simulation time

In [40]:
pauli_strings = [Vlasov_Hamiltonian_Paulis(n, k, α) for n in N_qubit];

In [41]:
L = [length(ps) for ps in pauli_strings]

8-element Vector{Int64}:
   14
   30
   62
  126
  254
  510
 1022
 2046

In [57]:
empirical_bounds = [ceil(Int,4*empirical_bound(n)) for n in N_qubit];

In [56]:
mimim_bounds = [4*minimize_bound(ϵ, 2, l) for l in L];

In [11]:
#Generate Full circuits
full_circuits = [Suzuki_Trotter_circuits_4thorder(ps, n, Tmax) for (ps, n) in zip(pauli_strings, N_qubit)]

8-element Vector{Circuit}:
 1: H(8)*S(2)*H(5)*H(7)*S(1)*S(3)*H(6)*S(4)
2: CNOT(5, 4)*CNOT(7, 3)*CNOT(6, 1)*CNOT(8, 2)
3: S(8)*S(2)*S(5)*S(7)*S(1)*S(3)*S(6)*S(4)
4: H(8)*H(2)*H(5)*H(7)*H(1)*H(3)*H(6)*H(4)
5: CNOT(3, 4)
6: Rz(0.004959204604512513)(4)
7: Rz(0.004959204604512513)(4)
8: CNOT(3, 4)
9: CNOT(2, 3)
10: CNOT(3, 7)
11: CNOT(7, 8)
12: Rz(0.007575310162317377)(8)
13: CNOT(7, 8)
14: CNOT(3, 7)
15: CNOT(2, 3)
16: CNOT(1, 2)
17: Rz(0.0036216909722333194)(2)
18: Rz(0.0036216909722333194)(2)
19: CNOT(1, 2)
20: CNOT(1, 4)
21: CNOT(4, 5)
22: CNOT(5, 6)
23: Rz(0.006402305614570701)(6)
24: CNOT(5, 6)
25: CNOT(4, 5)
26: CNOT(1, 4)
27: H(8)*H(2)*H(5)*H(7)*H(1)*H(3)*H(6)*H(4)
28: S†(8)*S†(2)*S†(5)*S†(7)*S†(1)*S†(3)*S†(6)*S†(4)
29: CNOT(5, 4)*CNOT(7, 3)*CNOT(6, 1)*CNOT(8, 2)
30: H(8)*S†(2)*H(5)*H(7)*S†(1)*S†(3)*H(6)*S†(4)
31: S(2)*H(5)*S(1)*S(3)*H(6)*H(4)
32: CNOT(5, 1)*CNOT(6, 2)*CNOT(4, 3)
33: S(2)*S(5)*S(1)*S(3)*S(6)*S(4)
34: H(2)*H(5)*H(1)*H(3)*H(6)*H(4)
35: CNOT(2, 3)
36: Rz(0.004049173603

In [3]:
ps = Vlasov_Hamiltonian_Paulis(2, k, α)
x = Suzuki_Trotter_circuits_4thorder(ps, 2, Tmax)
qasm = to_qasm(x)
open("./qasm_BBN/initial/plasma_hamsim_2.qasm", "w") do io
    write(io, string(qasm))
end

818

In [15]:
for (circuit, N) in zip(full_circuits, N_qubit)
    qasm = to_qasm(circuit)
    open("./qasm_BBN/initial/plasma_hamsim_$(N).qasm", "w") do io
        write(io, string(qasm))
    end
end

In [ ]:
# optimization, rotation synth run in bash

In [76]:
write("qasm_BBN/lattice_surgery/plasma_hamsim_8.json",
    run(pipeline(`cat qasm_BBN/ct/plasma_hamsim_8.qasm`, `staq_lattice_surgery`)))

LoadError: failed process: Process(`[4mstaq_lattice_surgery[24m`, ProcessSignaled(2)) [0]


In [74]:
for n = [8]
    write("./qasm_BBN/stats/plasma_hamsim_$(n)_CT.json", run(pipeline(`cat ./qasm_BBN/ct/plasma_hamsim_$n.qasm`, 
                 `python3 ../../util/rotation_synth/stats.py `)));
end

{
    "Number of qubits": 8,
    "Depth": 3395,
    "Number of gates": 4693,
    "Number of CNOTs": 144,
    "T gate count": 1708,
    "Non-clifford count": 0
}

In [75]:
1708/4693

0.3639463030044747